In [1]:
import sys, os

# Enable importing pysparky
sys.path.append(os.pardir)

In [2]:
import pyspark
from pyspark.sql import SparkSession, DataFrame, Column
from pyspark.sql import functions as F, types as T

print(pyspark.__version__)

spark = SparkSession.builder.getOrCreate()

3.5.2


24/10/17 13:00:45 WARN Utils: Your hostname, codespaces-0aafae resolves to a loopback address: 127.0.0.1; using 10.0.10.27 instead (on interface eth0)
24/10/17 13:00:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/17 13:00:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

data_pdf = pd.read_csv(
    "https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv",
    names=["first_name", "last_name", "address", "region", "code", "postcode"],
)
data_sdf = spark.createDataFrame(data_pdf)

24/10/17 13:01:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
data_sdf.show()

+--------------------+---------+--------------------+-----------+----+--------+
|          first_name|last_name|             address|     region|code|postcode|
+--------------------+---------+--------------------+-----------+----+--------+
|                John|      Doe|   120 jefferson st.|  Riverside|  NJ|    8075|
|                Jack| McGinnis|        220 hobo Av.|      Phila|  PA|    9119|
|       John "Da Man"|   Repici|   120 Jefferson St.|  Riverside|  NJ|    8075|
|             Stephen|    Tyler|7452 Terrace "At ...|   SomeTown|  SD|   91234|
|                 NaN| Blankman|                 NaN|   SomeTown|  SD|     298|
|Joan "the bone", ...|      Jet| 9th, at Terrace plc|Desert City|  CO|     123|
+--------------------+---------+--------------------+-----------+----+--------+



In [22]:
import functools
from pysparky.transformations import transforms, apply_cols
from pysparky.schema_ext import filter_columns_by_datatype

upper_cols_partial = functools.partial(apply_cols, col_func=F.upper)

In [23]:
pipeline = (
    (
        upper_cols_partial,
        {"cols": data_sdf.schema.filter_columns_by_datatype(T.StringType)},
    ),
    (lambda sdf: sdf.withColumn("postcode_plus_1", F.col("postcode") + 1), {}),
)

In [24]:
data_sdf.transforms(pipeline).show()

+--------------------+---------+--------------------+-----------+----+--------+---------------+
|          first_name|last_name|             address|     region|code|postcode|postcode_plus_1|
+--------------------+---------+--------------------+-----------+----+--------+---------------+
|                John|      Doe|   120 jefferson st.|  Riverside|  NJ|    8075|           8076|
|                Jack| McGinnis|        220 hobo Av.|      Phila|  PA|    9119|           9120|
|       John "Da Man"|   Repici|   120 Jefferson St.|  Riverside|  NJ|    8075|           8076|
|             Stephen|    Tyler|7452 Terrace "At ...|   SomeTown|  SD|   91234|          91235|
|                 NaN| Blankman|                 NaN|   SomeTown|  SD|     298|            299|
|Joan "the bone", ...|      Jet| 9th, at Terrace plc|Desert City|  CO|     123|            124|
+--------------------+---------+--------------------+-----------+----+--------+---------------+

